In [13]:
import pandas as pd
import calmap
import numpy as np
import os

In [14]:
from pathlib import Path
from os import environ, write

import re
from datetime import datetime, time
import datetime as dt
from collections import defaultdict

import requests

In [15]:
def load_data():
    if environ.get("USE_LOCAL_FILES"):
        paths = Path("extracted").rglob("*.csv")
        df_parts = []

        for path in sorted(paths):
            if path.parts[-1] < "2021-10-12T12-07-45.jsonl.csv":
                continue

            print("Reading for analysis:", path)

            with open(path, "r", encoding="utf-8") as fp:
                df_part = pd.read_csv(fp, delimiter=",", dtype=str)
                df_parts.append(df_part)

            # if df is None:
            #     df = df_part
            # else:
            #     df = df.append(df_part, ignore_index=True)

        df = pd.concat(df_parts, ignore_index=True)

    else:
        raise NotImplementedError("Remote data not yet implemented")
    
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors = 'coerce')
    df['date_of_proceeding'] = pd.to_datetime(df['date_of_proceeding'], errors = 'coerce')
    df['date_of_publication'] = pd.to_datetime(df['date_of_publication'], errors = 'coerce')

    return df

In [16]:
df = load_data()

Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-12T12-07-45.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-12T15-34-59.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-12T18-48-54.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-12T22-33-18.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-13T06-55-10.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-13T10-24-32.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-13T14-18-38.jsonl.csv
Reading for analysis: extracted/buckets/insolvenzenard/insolvenzbekanntmachungen-scraper/2021-10-13T18-03-26.jsonl.csv
Reading for analysis: extracted/buckets/insolven

In [17]:
df.drop_duplicates(subset ="description_hash", keep = "last", inplace = True)

In [19]:
# Funktion in_kind mit dem Argument/Wert df wird definiert

def ik_kind(df):
    ik_unternehmen = (df[df['kind'] == 'ik'])
    return ik_unternehmen

# Funktion def wird ausgeführt
data_ik = ik_kind(df)

In [9]:
data_nw_ik = data_ik[(data_ik.federal_state == 'Nordrhein-Westfalen')]

In [20]:
def ik_eröffnung(data_nw_ik):
    ik_eröffnet = (data_nw_ik[data_nw_ik['type_of_proceeding'] == 'Eröffnungen'])
    return ik_eröffnet

# Funktion def wird ausgeführt
data_nw_eröffnet = ik_eröffnung(data_nw_ik)

In [21]:
publication_frequency_eröffent = data_nw_eröffnet.groupby('date_of_publication').date_of_publication.count().reset_index(name="count")

In [22]:
publication_frequency_eröffent.to_csv('freq_ik_eröffnet.csv')

In [7]:
# Funktion in_kind mit dem Argument/Wert df wird definiert
def ik_kind(df):
    ik_privat = (df[df['kind'] == 'ik'])
    return ik_privat

# Funktion def wird ausgeführt
data_ik = ik_kind(df)

In [8]:
new_data_ik = data_ik.replace(['Nordrhein-Westfalen'], 'nw')

In [9]:
new_data_ik.federal_state.unique()

array(['ns', 'he', 'nw', 'hh', 'mv', 'rp', 'by', 'st', 'be', 'sl', 'th',
       'hb', 'sh', 'bb', 'sn', 'bw', 'Bayern', 'Rheinland-Pfalz',
       'Thüringen', 'Schleswig-Holstein', 'Sachsen-Anhalt', 'Sachsen',
       'Saarland', 'Niedersachsen', 'Mecklenburg-Vorpommern', 'Hessen',
       'Hamburg', 'Bremen', 'Brandenburg', 'Berlin', 'Baden-Württemberg'],
      dtype=object)

In [10]:
data_ik_nw = new_data_ik[(new_data_ik.federal_state == 'nw')]

In [24]:
data_ik_nw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192273 entries, 14 to 1099053
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _key                 15070 non-null   object        
 1   case_nr              192273 non-null  object        
 2   court                192273 non-null  object        
 3   date_of_birth        105656 non-null  datetime64[ns]
 4   date_of_proceeding   191990 non-null  datetime64[ns]
 5   date_of_publication  192273 non-null  datetime64[ns]
 6   federal_state        192273 non-null  object        
 7   file_name            170666 non-null  object        
 8   kind                 192273 non-null  object        
 9   title                170666 non-null  object        
 10  type_of_proceeding   192273 non-null  object        
 11  zipcode              192273 non-null  object        
 12  detail_form_name     21607 non-null   object        
 13  format      

In [26]:
#freq_in_nw_eröffnung['zipcode'] = freq_in_nw_eröffnung['zipcode'].astype(str)
data_ik_nw.loc['zipcode'] = data_ik_nw['zipcode'].astype(str)

<ipython-input-26-e06f5d88c44c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ik_nw['zipcode'] = data_ik_nw['zipcode'].astype(str)


### Anzahl der Vorgänge bei Unternehmensinsolvenzen von Januar bis Mitte September

In [13]:
#data_nw_in.type_of_proceeding.unique()
data_ik_nw['type_of_proceeding'].value_counts()

Restschuldbefreiung                                      81777
Eroeffnung_Insolvenzverfahren                            22534
Entscheidungen_im_Verfahren                              22475
Entscheidungen_im_Verfahren_Aufhebung_Einstellung        15683
Entscheidungen_im_Verfahren_mit_Termine                  13271
Verwalter_Treuhaender_Verteilungsverzeichnis             12623
Bestimmung_Termine                                        6481
Entscheidungen_im_Restschuldbefreiungsverfahren           6138
Eröffnungen                                               4140
Restschuldbefreiung_Erteilung_Versagung                   3050
Verteilungsverzeichnisse_(§_188_InsO)_d._Verw./Treuh.     2318
Verteilungsverzeichnis_Nachtragsvert_Loeschung             517
Sonstiges                                                  510
Sonstiges_ausserhalb_des_Verfahresn                        474
Entscheidungen_nach_Aufhebung_des_Verfahrens               158
Abweisungen_mangels_Masse                              

### Privatinsolvenz Eröffnungen von 01.01. - 10.09. in NRW 

In [17]:
def ik_eröffnung(data_ik_nw):
    ik_eröffnet = (data_ik_nw[data_ik_nw['type_of_proceeding'] == 'Eroeffnung_Insolvenzverfahren'])
    return ik_eröffnet

# Funktion def wird ausgeführt
data_nw_eröffnet = ik_eröffnung(data_ik_nw)

In [22]:
data_nw_eröffnet.sample(5)

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
191042,NaN,070f_IK00059_21,agkoeln,1962-10-27,2021-02-05,2021-02-08,nw,gerichte/nw/agkoeln/21/070f_IK00059_21/2021_02...,ik,"Kornett, Daniel, Wiehl, 70f IK 59/21",Eroeffnung_Insolvenzverfahren,51674,NaN,NaN,NaN,NaN,NaN
803275,NaN,0166_IK00093_21,agessen,1994-12-26,2021-05-11,2021-05-25,nw,gerichte/nw/agessen/21/0166_IK00093_21/2021_05...,ik,"Reh, Philipp, Essen, 166 IK 93/21",Eroeffnung_Insolvenzverfahren,45307,NaN,NaN,NaN,NaN,NaN
626973,NaN,0096_IK00064_21,agbonn,1991-09-13,2021-04-07,2021-04-13,nw,gerichte/nw/agbonn/21/0096_IK00064_21/2021_04_...,ik,"Fabri, Josef Michael, Much, 96 IK 64/21",Eroeffnung_Insolvenzverfahren,53804,NaN,NaN,NaN,NaN,NaN
688659,NaN,070d_IK00039_21,agkoeln,1956-05-01,2021-04-30,2021-05-04,nw,gerichte/nw/agkoeln/21/070d_IK00039_21/2021_05...,ik,"Hommes, Hannelore Annemarie, Köln, 70d IK 39/21",Eroeffnung_Insolvenzverfahren,51103,NaN,NaN,NaN,NaN,NaN
857461,NaN,070f_IK00139_21,agkoeln,1988-12-07,2021-05-21,2021-05-25,nw,gerichte/nw/agkoeln/21/070f_IK00139_21/2021_05...,ik,"Junker, Stefanie, Wesseling, 70f IK 139/21",Eroeffnung_Insolvenzverfahren,50389,NaN,NaN,NaN,NaN,NaN


#### Groupby date_of-Publication

In [27]:
freq_ik_nw_eröffnung = data_nw_eröffnet.groupby(['date_of_publication'], as_index=False).count() 
freq_ik_nw_eröffnung.sample(5)

,date_of_publication,_key,case_nr,court,date_of_birth,date_of_proceeding,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
100,2021-05-20,0,186,186,182,186,186,186,186,186,186,186,0,0,0,0,0
45,2021-03-05,0,148,148,146,148,148,148,148,148,148,148,0,0,0,0,0
84,2021-04-28,0,164,164,162,164,164,164,164,164,164,164,0,0,0,0,0
9,2021-01-14,94,94,94,94,94,94,94,94,94,94,94,0,0,0,0,0
103,2021-05-25,0,136,136,136,136,136,136,136,136,136,136,0,0,0,0,0


In [29]:
freq_ik_nw_eröffnung.to_csv('freq_ik_nw_eröffnung_public.csv')

#### Groupby date_of_birth

In [31]:
freq_ik_nw_eröffnung_birth = data_nw_eröffnet.groupby(['date_of_birth'], as_index=False).count() 
freq_ik_nw_eröffnung_birth.sample(5)

,date_of_birth,_key,case_nr,court,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint
7509,1994-08-08,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
6773,1990-12-06,0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0
6168,1988-03-08,0,6,6,6,6,6,6,6,6,6,6,0,0,0,0,0
1935,1966-02-20,1,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0
739,1957-07-20,0,4,4,4,4,4,4,4,4,4,4,0,0,0,0,0


In [33]:
freq_ik_nw_eröffnung_birth.to_csv('freq_ik_nw_eröffnung_birth.csv')

#### Restschuld Privatunternehmen

In [38]:
def ik_restschuld(data_ik_nw):
    ik_rest = (data_ik_nw[data_ik_nw['type_of_proceeding'] == 'Restschuldbefreiung'])
    return ik_rest

# Funktion def wird ausgeführt
restschuld_ik = ik_restschuld(data_ik_nw)

In [39]:
restschuld_ik.describe()

,register
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN
